<h4>Imports</h4>

In [50]:
#NOTE: autoreload is not recursive, only direct imports are reloaded
%load_ext autoreload
%autoreload 2
from data_translator import TestTranslator, EncodeForGA, GAToScheduleTranslator
from hybrid_solution_data_loader import get_data
from models import Order, SimulationEnvironment
from solver import GASolver, PSOSolver, GreedyAgentSolver
from visualize import visualize_schedule_demo
import objective_function

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h4>Read from Source</h4>

In [12]:
n_workstations, recipes, operation_times = get_data(0)
recipies, workstations, resources, tasks, _ = TestTranslator().translate(n_workstations, recipes, operation_times)

env = SimulationEnvironment(workstations, tasks, resources, recipies)

<h4>Create Orders</h4>

In [13]:
earliest_slot = 0
last_slot = 100
recipe_orders = [0, 1, 2, 3, 0, 3, 2, 1, 0, 3] # for dataset 0
orders = []
o_id = 0
for order in recipe_orders:
    orders.append(Order(o_id, 0, last_slot, last_slot, [order], 100, 50, False, 0, False, 500)) # for now: use resources to select recipe
    o_id = o_id + 1


<h4>Translate for Solver</h4>

In [14]:
# translate datamodel to encoding
encoder = EncodeForGA()
# TODO: alternatives can be completely removed (TEST first)
values, durations, all_jobs, alternatives = encoder.translate(env, orders) # encoding, duration lookup table, list of all jobs used (probably not needed), possible alternatives for each job


<h4>Run Solver</h4>

GA

In [104]:
#NOTE: Best makespan so far: 26
crossover = 'two_points' #NOTE: available in PyGAD: 'two_points', 'single_point', 'uniform', 'scattered'
selection = 'rws' #NOTE: available in PyGAD: 'sss' (steady state selection', 'rws' (roulette wheel), 'sus' (stocastic universal selection), 'rank' (rank selection), 'random' (random selection), 'tournament' (tournament selection)
population_size = 50
offspring_amount = 100
max_generations = 2000

solver = GASolver(values, durations, all_jobs, alternatives, env, orders)
solver.initialize(earliest_slot, last_slot, population_size, offspring_amount, max_generations, crossover=crossover, selection=selection)
solver.run()

Done


In [105]:
# just for comparison, so the optimizers don't have to run over and over again
ga_result = solver.get_best()
ga_fitness = solver.get_best_fitness()

PSO (don't run, currently under construction)

In [106]:
#NOTE: no best makespan yet
solver = PSOSolver(values, durations, all_jobs, env, orders)

solver.initialize(earliest_slot, last_slot, c1=0.5, c2=0.3, w=0.5, max_iter=5000)
solver.run()

Stopping search: maximum iterations reached --> 5000
However, the optimization couldn't find a feasible design. Sorry


In [107]:
# just for comparison, so the optimizers don't have to run over and over again
pso_result = solver.get_best()
pso_fitness = solver.get_best_fitness()

Greedy Agent (don't run, currently under construction)

In [133]:
#NOTE: Best makespan so far: 26
solver = GreedyAgentSolver(values, durations, all_jobs, env, orders)
solver.run()

In [134]:
# just for comparison, so the optimizers don't have to run over and over again
greedy_result = solver.get_best()
greedy_fitness = solver.get_best_fitness()

<h4>Result Output</h4>

In [141]:
result = solver.get_best()
fitness = solver.get_best_fitness()
#result = greedy_result
#fitness = greedy_fitness
#result = pso_result
#fitness = pso_fitness
#result = ga_result
#fitness = ga_fitness

print("Parameters of the best solution : {solution}".format(solution=result))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(fitness)))
fitness_history = solver.assignments_best
average_fitness_history = solver.average_assignments


Parameters of the best solution : [ 4  0  1  2  3  6  1  6  1 11  4 17  2  0  2  6  3 10  3 12  3 13  3 17
  2  8  1 17  1 21  2 12  1 26  0  0  2 14  3 18  3 20  0  9  0 11  4 22
  0 16  0 18  0 23  4  2  3 21]
Fitness value of the best solution = 27


In [142]:
import matplotlib.pyplot as plt
if not len(fitness_history) == 0 and not len(average_fitness_history) == 0:
    x = list(range(0, len(fitness_history)))
    plt.plot(x, fitness_history)
    plt.plot(x, average_fitness_history)
    plt.title('Fitness History')
    plt.legend(['Best Known Fitness', 'Average Fitness'])
    plt.xlabel = 'Generation'
    plt.ylabel = 'Fitness'
    plt.show()

<h4>(Optional) Postprocessing</h4>

<h4>Translate to Schedule</h4>

In [143]:
schedule = GAToScheduleTranslator().translate(result, all_jobs, env, orders)

<h4>Visualize Schedule</h4>

In [144]:
visualize_schedule_demo(schedule, env, orders)

<h4>Evaluate Schedule</h4>

In [145]:
#NOTE: the objective functions in this case assume the solution is feasible, no additional checks
#NOTE: currently the only objective function used during optimization is makespan
makespan, tardiness, deviation, idle_time, profit = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {makespan}\nTardiness: {tardiness}\nDeviation: {deviation}\nIdle Time: {idle_time}\nProfit: {profit}')

Fitness Values:
Makespan: 27
Tardiness: 0
Deviation: 787
Idle Time: 11
Profit: 5000
